In [ ]:
import pytearcat as pt
import sympy as sp
import numpy as np
from sympy import *
init_printing(use_unicode=True)

In [ ]:
class Equation:
    """
    This class represents an equation in the teleparallel gravity model.
    """

    def __init__(self, coordinates, functions, constant, line_dif, *args):
        """
        Initializes the Equation object with the given parameters.

        Parameters:
        coordinates (str): The coordinates to be used.
        functions (list): A list of functions to be used.
        constant (float): A constant value to be used in the equation.
        line_dif (str): The line element differential.
        *args: Additional arguments.
        """
        ...
        self.coordinates = coordinates
        self.functions = functions
        self.constant = constant
        self.line_dif = line_dif
        self.args = args
        
        self.t,self.r,self.theta,self.phi=pt.coords(self.coordinates)
        
        if self.functions is not None:
            for function in self.functions:
                name, variables = function
                exec(f"{name} = pt.fun('{name}', '{variables}')")
        
        if self.constant is not None:
            for constant_list in self.constant:
                exec(f"{constant_list} = pt.con('{constant_list}')")
        
        self.KD = pt.kdelta()
        
        ds=self.line_dif
        self.g=pt.metric(ds)
        
    

    def constants(self):
        """
        Calculates and returns the constants C1, C2, C3, C4, C5, C6 based on the metric tensor.

        Returns:
        tuple: A tuple containing the constants C1, C2, C3, C4, C5, C6.
        """
        ...
        
        g11 = self.g.tensor[0][0][0]
        g22 = self.g.tensor[0][1][1]
        g33 = self.g.tensor[0][2][2]
        
        C1 = sp.simplify(sp.sqrt(-1*g11))
        C2 = 0
        C3 = 0
        C4 = sp.simplify(sp.sqrt(g22))
        C5 = sp.simplify(sp.sqrt(g33)) 
        C6 = 0

        C1 = C1.replace(Abs, lambda arg: arg)
        C4 = C4.replace(Abs, lambda arg: arg)
        C5 = C5.replace(Abs, lambda arg: arg)

        
        return C1, C2, C3, C4, C5, C6

    def tetrad(self, cond):
        """
        Calculates and returns the tetrad based on the constants and the coordinates.

        Parameters:
        cond (bool): A condition to display the tetrad.

        Returns:
        object: The tetrad.
        """
        ...
        
        C1, C2, C3, C4, C5, C6 = self.constants()
        e=pt.ten('e',2)
        e.assign([[C1,C2,0,0],[C3*sp.sin(self.theta)*sp.cos(self.phi),C4*sp.sin(self.theta)*sp.cos(self.phi),C5*sp.cos(self.theta)*sp.cos(self.phi)-C6*sp.sin(self.phi),-sp.sin(self.theta)*(C5*sp.sin(self.phi)+C6*sp.cos(self.theta)*sp.cos(self.phi))],
                [C3*sp.sin(self.theta)*sp.sin(self.phi),C4*sp.sin(self.theta)*sp.sin(self.phi),C5*sp.cos(self.theta)*sp.sin(self.phi)+C6*sp.cos(self.phi),sp.sin(self.theta)*(C5*sp.cos(self.phi)-C6*sp.cos(self.theta)*sp.sin(self.phi))],
                [C3*sp.cos(self.theta),C4*sp.cos(self.theta),-C5*sp.sin(self.theta),C6*sp.sin(self.theta)**2]],'^A,_mu')
        
        if cond == True:
            e.display('^,_')
        
        return e

    def inv_tetrad(self, cond):
        """
        Calculates and returns the inverse tetrad based on the tetrad.

        Parameters:
        cond (bool): A condition to display the inverse tetrad.

        Returns:
        object: The inverse tetrad.
        """
        ...
        
        M = self.tetrad(False)
        M = Matrix(M.tensor[2])
        M_inverse = M.inv()
        M_lol=M_inverse.tolist()
        E=pt.ten('E',2)
        E.assign(M_lol,'^mu,_A')
        E.simplify()
        
        if cond == True:
            E.display('^,_')
        
        return E

    def mink(self, cond):
        """
        Returns the Minkowski metric tensor.

        Parameters:
        cond (bool): A condition to display the Minkowski metric tensor.

        Returns:
        object: The Minkowski metric tensor.
        """
        ...
        eta=pt.ten('eta',2)
        eta.assign([[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],'_A,_B')
        
        if cond == True:
            eta.display('_,_')
        
        return eta
        

    def mink_inv(self, cond):
        """
        Returns the inverse Minkowski metric tensor.

        Parameters:
        cond (bool): A condition to display the inverse Minkowski metric tensor.

        Returns:
        object: The inverse Minkowski metric tensor.
        """
        ...
        
        eta=pt.ten('eta',2)
        eta.assign([[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],'^A,^B')
       
        
        if cond == True:
            eta.display('^,^')
        
        return eta

    def metric_check(self, cond):
        """
        Checks the metric by calculating the product of the Minkowski metric tensor and the tetrad.

        Parameters:
        cond (bool): A condition to display the result.

        Returns:
        object: The result of the check.
        """
        ...
        
        eta = self.mink(False)
        e = self.tetrad(False)
        
        ge = pt.ten('ge',2)
        val1 = eta("_A,_B")*e("^A,_mu")*e("^B,_nu")
        ge.assign(val1,"_mu,_nu")
        ge.simplify()
        
        if cond == True:
            ge.display('_,_')
            
        return ge
        
    def pretorsion(self, cond):
        """
        Calculates and returns the pretorsion tensor.

        Parameters:
        cond (bool): A condition to display the pretorsion tensor.

        Returns:
        object: The pretorsion tensor.
        """
        ...
        e = self.tetrad(False)
        torsion = pt.ten('torsion',3)
        val3 = pt.D(e('^A,_nu'),'_mu')-pt.D(e('^A,_mu'),'_nu')
        torsion.assign(val3, "^A,_mu,_nu")
        torsion.simplify()
        
        if cond == True:
            torsion.display('^,_,_')
            
        return torsion

    def torsion(self, cond, indices):
        """
        Calculates and returns the torsion tensor.

        Parameters:
        cond (bool): A condition to display the torsion tensor.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The torsion tensor.
        """
        ...
        
        torsion = self.pretorsion(False)
        E = self.inv_tetrad(False)
        torsiongreek = pt.ten('torsiongreek',3)
        val4 = torsion("^A,_mu,_nu")*E("^alpha,_A")
        torsiongreek.assign(val4, "^alpha,_mu,_nu")
        torsiongreek.simplify()
        torsiongreek.complete('^,_,_')
        
        if cond == True:
            torsiongreek.display(indices)
        
        return torsiongreek
    
    
    def contorsion(self, cond, indices):
        """
        Calculates and returns the contorsion tensor.

        Parameters:
        cond (bool): A condition to display the contorsion tensor.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The contorsion tensor.
        """
        ...
        
        torsiongreek = self.torsion(False, False)
        contorsiongreeknew = pt.ten('contorsiongreeknew',3)
        val6 = 1/2*(torsiongreek("_rho,^nu,_mu")+torsiongreek("_mu,^nu,_rho")-torsiongreek("^nu,_rho,_mu"))
        contorsiongreeknew.assign(val6, "^nu,_rho,_mu")
        contorsiongreeknew.simplify()
        contorsiongreeknew.complete('^,_,_')
        
        if cond == True:
            contorsiongreeknew.display(indices)
            
        return contorsiongreeknew

    def superpotential(self, cond, indices):
        """
        Calculates and returns the superpotential tensor.

        Parameters:
        cond (bool): A condition to display the superpotential tensor.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The superpotential tensor.
        """
        ...
        
        contorsiongreeknew = self.contorsion(False, False)
        torsiongreek = self.torsion(False, False)
        superpotential = pt.ten('superpotential',3)
        valchido = 1/2*contorsiongreeknew("^mu,^nu,_rho")-1/2*self.KD("^mu,_rho")*
        torsiongreek("_sigma,^sigma,^nu")+1/2*self.KD("^nu,_rho")*torsiongreek("_sigma,^sigma,^mu")
        superpotential.assign(valchido, "_rho,^mu,^nu")
        superpotential.simplify()
        superpotential.complete('_,^,^')
        
        if cond == True:
            superpotential.display(indices)
            
        return superpotential

    def torsionscalar(self):
        """
        Calculates and returns the torsion scalar.

        Returns:
        object: The torsion scalar.
        """
        ...
        
        superpotential = self.superpotential(False, False)
        torsiongreek = self.torsion(False, False)
        ge = self.metric_check(False)
        t_scalar=superpotential("^mu,^beta,^gamma")*torsiongreek("^alpha,_beta,_gamma")*ge("_alpha,_mu")

        return t_scalar

    def energy_momentum(self, cond, indices):
        """
        Calculates and returns the energy-momentum tensor.

        Parameters:
        cond (bool): A condition to display the energy-momentum tensor.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The energy-momentum tensor.
        """
        ...
        
        U=pt.ten('U',1)
        U.assign([1,0,0,0],'^')
        U.display('^')
        rho = pt.fun('rho','t')
        P = pt.fun('P','t')
        Theta = pt.ten('Theta',2)
        val11 = ((rho+P)*U("^mu")*U("^nu")+P*self.g("^mu,^nu"))
        Theta.assign(val11,"^mu,^nu")
        Theta.complete('^,^')
        
        if cond == True:
            Theta.display(indices)
            
        return Theta

    def determinant(self):
        """
        Calculates and returns the determinant of the tetrad.

        Returns:
        object: The determinant of the tetrad.
        """
        ...
        
        M = self.tetrad(False)
        d = Matrix(M.tensor[2])
        det = d.det()
        dett = sp.simplify(det)
        
        return dett

    def ricci_scalar(self):
        """
        Calculates and returns the Ricci scalar.

        Returns:
        object: The Ricci scalar.
        """
        ...
        
        Ri=pt.ricci()
        Ris=pt.riccis()
        
        return Ris 

    def boundary(self):
        """
        Calculates and returns the boundary term.

        Returns:
        object: The boundary term.
        """
        ...
        
        dett = self.determinant()
        torsiongreek = self.torsion(False, False)
        Bterm=2/dett*pt.D(dett*torsiongreek('^nu,_nu,^mu'),'_mu')
        return Bterm

    def Teleparallel_Equivalent(self, cond, indices):
        """
        Calculates and returns the Teleparallel Equivalent of General Relativity (TEGR) field equations.

        Parameters:
        cond (bool): A condition to display the TEGR field equations.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The TEGR field equations.
        """
        ...
        
        dett = self.determinant()
        superpotential = self.superpotential(False, False)
        torsiongreek = self.torsion(False, False)
        e = self.tetrad(False)
        E = self.inv_tetrad(False)
        torsion_scalar = self.torsionscalar()
        
        superpotentialequation = pt.ten('superpotentialequation',3)
        valsuperpotential = superpotential("_nu,^mu,^sigma")*E("^nu,_A")
        superpotentialequation.assign(valsuperpotential, "_A,^mu,^sigma")
        superpotentialequation.simplify()
        
        term1 = pt.ten('term1',2)
        valterm1 = 2*1/dett*e("^A,_nu")*pt.D(dett*superpotentialequation("_A,^mu,^sigma"),'_sigma')
        term1.assign(valterm1, "_nu,^mu")
        term1.complete('_,^')

        term2 = pt.ten('term2',2)
        valterm2 = 2*torsiongreek("^sigma,_nu,_beta")*superpotential("_sigma,^mu,^nu")
        term2.assign(valterm2, "_beta,^mu")
        term2.complete('_,^')

        term3 = pt.ten('term3',2)
        valterm3 = 1/2*E("^mu,_A")*e('^A,_nu')*torsion_scalar
        term3.assign(valterm3, "^mu,_nu")
        term3.complete('^,_')

        Teleparallel = pt.ten('Teleparallel',2)
        valtel = term1("_nu,^mu")+term2("_nu,^mu")+term3("_nu,^mu")-K**2*Theta("_nu,^mu")
        Teleparallel.assign(valtel, "_nu,^mu")
        Teleparallel.complete('_,^')
        
        if cond == True:
            Teleparallel.display(indices, aslist=True)
            
        return Teleparallel

    def func(self, cond, indices):
        """
        Defines and returns the functions and their gradients.

        Parameters:
        cond (bool): A condition to display the gradients.
        indices (str): The indices to be used in the tensor.

        Returns:
        tuple: A tuple containing the functions and their gradients.
        """
        ...
        
        FT = pt.fun("FT","t,r")
        FB = pt.fun("FB","t,r")
        F = pt.fun("F", "t,r")
        B = pt.fun("B", "t,r")
        
        gradFT = pt.ten('gradFT', 1)
        gradFT.assign([Derivative(FT,self.t),Derivative(FT,self.r),0,0],'_')
        
        gradFT.complete('_')
        
        if cond == True:
            gradFT.display(indices)

        
        
        gradFB = pt.ten('gradFB', 1)
        gradFB.assign([Derivative(FB,self.t),Derivative(FB,self.r),0,0],'_')
        gradFB.complete('_')
        if cond == True:
            gradFB.display(indices)
        
        return F, B, FT, FB, gradFT, gradFB
        
    def terms(self, cond, indices):
        """
        Calculates and returns the terms used in the Teleparallel Equivalent of General Relativity (TEGR) field equations.

        Parameters:
        cond (bool): A condition to display the terms.
        indices (str): The indices to be used in the tensor.

        Returns:
        tuple: A tuple containing the terms.
        """
        ...
        
        F, B, FT, FB, gradFT, gradFB = self.func(False, False)
        dett = self.determinant()
        superpotential = self.superpotential(False, False)
        torsiongreek = self.torsion(False, False)
        e = self.tetrad(False)
        E = self.inv_tetrad(False)

        pt.christoffel()
        
        
        term1 = pt.ten('term1',2)
        valterm1 = 2*self.KD("_nu,^mu")*pt.C(gradFB("^delta"),"_delta")-2*self.g("^lambda,^mu")*pt.C(gradFB("_nu"),"_lambda")
        term1.assign(valterm1, "_nu,^mu")
        term1.complete('_,^')
        
        if cond == True:
            print('First term')
            term1.display(indices, aslist=True)
        
        term2 = pt.ten('term2',2)
        valterm2 = self.KD("_nu,^mu")*(B*FB-F)
        term2.assign(valterm2, "_nu,^mu")
        term2.complete('_,^')
        
        if cond == True:
            print('Second term')
            term2.display(indices, aslist=True)
        
        term3 = pt.ten('term3',2)
        valterm3 = 4*(gradFB("_rho")+gradFT("_rho"))*superpotential("_nu,^rho,^mu")
        term3.assign(valterm3, "_nu,^mu")
        term3.complete('_,^')
        
        if cond == True:
            print('Third term')
            term3.display(indices, aslist=True)
        
        term4 = pt.ten('term4',2)
        valterm4 = 4*e("^A,_beta")*pt.D(dett*superpotential("_beta,^nu,^mu")*E("^beta,_A"),"_nu")*1/dett*FT
        term4.assign(valterm4, "_beta,^mu")
        term4.complete('_,^')
        
        if cond == True:
            print('Fourth term')
            term4.display(indices, aslist=True)
        
        term5 = pt.ten('term5',2)
        valterm5 = (-1)*4*FT*torsiongreek("^sigma,_nu,_beta")*superpotential("_sigma,^mu,^nu")
        term5.assign(valterm5, "_beta,^mu")
        term5.complete('_,^')
        
        if cond == True:
            print('Fifth term')
            term5.display(indices, aslist=True)
            
        return term1, term2, term3, term4, term5

    def teleparallel_FTB(self, cond, indices):
        """
        Calculates and returns the TEGR field equations with the terms calculated in the `terms` method.

        Parameters:
        cond (bool): A condition to display the TEGR field equations.
        indices (str): The indices to be used in the tensor.

        Returns:
        object: The TEGR field equations.
        """
        ...
        
        term1, term2, term3, term4, term5 = self.terms(False, False)
        Theta = self.energy_momentum(False, False)
        
        K = pt.con('K')
        
        Teleparallel = pt.ten('Teleparallel',2)
        valtel = term1("_nu,^mu")+term2("_nu,^mu")+term3("_nu,^mu")+term4("_nu,^mu")+term5("_nu,^mu")-2*K**2*Theta("_nu,^mu")
        Teleparallel.assign(valtel, "_nu,^mu")
        Teleparallel.complete('_,^')
        
        if cond == True:
            print('Teleparallel field equations')
            Teleparallel.display(indices, aslist=True)
            
        return Teleparallel



In [ ]:
McVittie_metric = Equation('t,r,theta,phi',[['mu','t,r'],['a','t']] ,None,'ds2=-((1-mu)**2)/((1+mu)**2)*dt**2+a**2*(1+mu)**4*dr**2+a**2*r**2*(1+mu)**4*dtheta**2+a**2*r**2*(1+mu)**4*sin(theta)**2*dphi**2')